In [44]:
import pandas as pd
import sqlite3
import pandas as pd


# Connect to the SQLite database
db_path = '../data/WARP.db'
conn = sqlite3.connect(db_path)

# Read the raw_NED_preds table into a DataFrame
df_NED_preds_processed = pd.read_sql_query("SELECT * FROM raw_NED_preds", conn)

# Close the database connection
conn.close()


import pandas as pd

# Zorg dat je datumkolommen correct zijn
df_NED_preds_processed['validfrom'] = pd.to_datetime(df_NED_preds_processed['validfrom'])
df_NED_preds_processed['validto'] = pd.to_datetime(df_NED_preds_processed['validto'])
df_NED_preds_processed = df_NED_preds_processed.set_index('lastupdate')
pd.to_datetime(df_NED_preds_processed.index)
df_NED_preds_processed.index = pd.to_datetime(df_NED_preds_processed.index)
df_NED_preds_processed.index



# Calculate rows where validfrom is earlier than lastupdate
mask = df_NED_preds_processed['validfrom'] >= df_NED_preds_processed.index
# Keep only rows that match this condition
print(f"Number of rows where validfrom is earlier than lastupdate: {(~mask).sum()}")



# Verwijder irrelevante kolommen
df_NED_preds_processed = df_NED_preds_processed.drop(columns=[
    '@id', '@type', 'id', 'point', 'granularity', 'granularitytimezone', 'activity', 'classification', 'capacity','percentage','emission','emissionfactor','validfrom',])
# remove rows with validto dates earlier than 2025-04-09 01:00:00+00:00
df_NED_preds_processed = df_NED_preds_processed[df_NED_preds_processed['validto'] > '2025-04-09 01:00:00+00:00']
# Rename current_datetime to 'fetch_timestamp'
df_NED_preds_processed = df_NED_preds_processed.rename(columns={'current_datetime': 'fetch_timestamp'})
#
df_NED_preds_processed['type'] = df_NED_preds_processed['type'].str.replace('/v1/types/', 'PRED_Type_')

print(df_NED_preds_processed.head())

# Save the processed DataFrame to a new SQLite database
#output_db_path = '../data/WARP.db'
#conn = sqlite3.connect(output_db_path)
#df_NED_preds_processed.to_sql('processed_NED_preds', conn, if_exists='replace', index=True)
#conn.close()

Number of rows where validfrom is earlier than lastupdate: 8229
                                  type  volume                   validto  \
lastupdate                                                                 
2025-04-04 22:45:27+00:00  PRED_Type_1  172140 2025-04-09 02:00:00+00:00   
2025-04-04 22:45:27+00:00  PRED_Type_1  150140 2025-04-09 03:00:00+00:00   
2025-04-04 22:45:27+00:00  PRED_Type_1  138338 2025-04-09 04:00:00+00:00   
2025-04-04 22:45:27+00:00  PRED_Type_1  182971 2025-04-09 05:00:00+00:00   
2025-04-04 22:45:27+00:00  PRED_Type_1  254562 2025-04-09 06:00:00+00:00   

                               fetch_timestamp  
lastupdate                                      
2025-04-04 22:45:27+00:00  2025-04-04 23:20:09  
2025-04-04 22:45:27+00:00  2025-04-04 23:20:09  
2025-04-04 22:45:27+00:00  2025-04-04 23:20:09  
2025-04-04 22:45:27+00:00  2025-04-04 23:20:09  
2025-04-04 22:45:27+00:00  2025-04-04 23:20:09  


In [45]:
# transforming forecasted electricity production data (e.g. from NED.nl) from long format to wide format, with explicit forecast horizon labels like pred_volume_type1_d1, pred_volume_type1_d2, etc.
df_NED_preds_processed["day_ahead"] = (df_NED_preds_processed["validto"].dt.normalize() - pd.to_datetime(df_NED_preds_processed.index).normalize()).dt.days
df_NED_preds_processed["pred_column"] = "volume_" + df_NED_preds_processed["type"].str.lower() + "_d" + df_NED_preds_processed["day_ahead"].astype(str)
df_NED_preds_processed["day_ahead"] = (df_NED_preds_processed["validto"].dt.normalize() - pd.to_datetime(df_NED_preds_processed.index).normalize()).dt.days
df_NED_preds_processed["pred_column"] = "volume_" + df_NED_preds_processed["type"].str.lower() + "_d" + df_NED_preds_processed["day_ahead"].astype(str)
# Group by validto and pred_column, taking the last value in case of duplicates
# dealing with duplicates
# x.dropna() removes NaNs, .iloc[-1] picks the last non-NaN value, if all values are NaN, it returns np.nan:
df_wide = df_NED_preds_processed.pivot_table(
    index="validto",
    columns="pred_column",
    values="volume",
    aggfunc=lambda x: x.dropna().iloc[-1] if not x.dropna().empty else np.nan
)

# Sort df_wide by validto index and display head and tail
print(df_wide.sort_index().tail(5))
print("\nTail of df_wide (last 5 rows):")



pred_column                volume_pred_type_17_d0  volume_pred_type_17_d1  \
validto                                                                     
2025-05-12 06:00:00+00:00                     NaN                     NaN   
2025-05-12 07:00:00+00:00                     NaN                     NaN   
2025-05-12 08:00:00+00:00                     NaN                     NaN   
2025-05-12 09:00:00+00:00                     NaN                     NaN   
2025-05-12 10:00:00+00:00                     NaN                     NaN   

pred_column                volume_pred_type_17_d2  volume_pred_type_17_d3  \
validto                                                                     
2025-05-12 06:00:00+00:00                     NaN                     NaN   
2025-05-12 07:00:00+00:00                     NaN                     NaN   
2025-05-12 08:00:00+00:00                     NaN                     NaN   
2025-05-12 09:00:00+00:00                     NaN                     NaN  

In [20]:

pivot_table = pd.pivot_table(
    df_NED_preds_processed,
    values='volume',
    index='lastupdate',
    columns='type',
    aggfunc='count'
)

# Display the pivot table
print(pivot_table)



type                       PRED_Type_1  PRED_Type_17  PRED_Type_2  \
lastupdate                                                          
2025-04-01 23:00:00+00:00         26.0          26.0         26.0   
2025-04-02 07:00:00+00:00        174.0         174.0        174.0   
2025-04-02 08:00:00+00:00        175.0         175.0        175.0   
2025-04-02 09:00:00+00:00        176.0         176.0        176.0   
2025-04-02 10:00:00+00:00        177.0         177.0        177.0   
...                                ...           ...          ...   
2025-05-03 11:00:00+00:00        178.0         178.0        178.0   
2025-05-03 23:00:00+00:00        170.0         170.0        170.0   
2025-05-04 11:00:00+00:00        178.0         178.0        178.0   
2025-05-04 23:00:00+00:00        170.0         170.0        170.0   
2025-05-05 11:00:00+00:00        178.0         178.0        178.0   

type                       PRED_Type_20  PRED_Type_21  PRED_Type_26  \
lastupdate                     

In [5]:
print(df_NED_preds_processed.head)

<bound method NDFrame.head of                          validto                lastupdate  /v1/types/1  \
0      2025-04-01 23:00:00+00:00 2025-04-01 23:45:27+00:00     25714039   
1      2025-04-01 23:00:00+00:00 2025-04-01 23:45:59+00:00            0   
2      2025-04-01 23:00:00+00:00 2025-04-01 23:46:22+00:00            0   
3      2025-04-01 23:00:00+00:00 2025-04-01 23:46:24+00:00            0   
4      2025-04-01 23:00:00+00:00 2025-04-01 23:46:25+00:00            0   
...                          ...                       ...          ...   
125992 2025-05-12 10:00:00+00:00 2025-05-05 11:46:18+00:00            0   
125993 2025-05-12 10:00:00+00:00 2025-05-05 11:46:19+00:00            0   
125994 2025-05-12 10:00:00+00:00 2025-05-05 11:46:21+00:00            0   
125995 2025-05-12 10:00:00+00:00 2025-05-05 11:46:24+00:00            0   
125996 2025-05-12 10:00:00+00:00 2025-05-05 11:46:34+00:00            0   

        /v1/types/17  /v1/types/2  /v1/types/20  /v1/types/21  /v1/ty